## Обзор данных

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
from scipy import stats as st

Для исследования предоставлены несколько датасетов, рассмотрим каждый из них.

In [2]:
#открываем файл с данными
users = pd.read_csv('..../users.csv)

#вывод первых 5 строк таблицы
users.head()

SyntaxError: EOL while scanning string literal (3675466726.py, line 2)

In [ ]:
#вывод общей информации по таблице
users.info()

**Таблица `users`** состоит из 8 столбцов с типами данных `int64`(2), `object`(6) и содержит информацию о 500 пользователях:
* `user_id` — уникальный идентификатор пользователя;
* `first_name` — имя пользователя;
* `last_name` — фамилия пользователя;
* `age` — возраст пользователя (годы);
* `reg_date` — дата подключения тарифа (день, месяц, год);
* `churn_date` — дата прекращения пользования тарифом;
* `city` — город проживания пользователя;
* `tariff` — название тарифного плана.

В столбце `churn_date` всего 38 строк заполнено (если значение пропущено, то тариф ещё действовал на момент выгрузки данных), т. е. наличие пропусков в этом столбце является нормальным, для удобства их можно заменить на `-`.

In [ ]:
#открываем файл с данными
calls = pd.read_csv('/datasets/calls.csv')

#вывод первых 5 строк таблицы
calls.head()

In [ ]:
calls.info()

**Таблица `calls`** состоит из 4 столбцов с типами данных `float64`(1), `int64`(1), `object`(2) и содержит информацию о 202607 звонках:
* `id` — уникальный номер звонка;
* `call_date` — дата звонка (день, месяц, год);
* `duration` — длительность звонка (в минутах);
* `user_id` — идентификатор пользователя, сделавшего звонок.

In [ ]:
#открываем файл с данными
messages = pd.read_csv('/datasets/messages.csv')

#вывод первых 5 строк таблицы
messages.head()

In [ ]:
messages.info()

**Таблица `messages`** состоит из 3 столбцов с типами данных `int64`(1), `object(2)` и содержит информацию о 123036 сообщениях:
* `id` — уникальный номер сообщения;
* `message_date` — дата сообщения (день, месяц, год);
* `user_id` — идентификатор пользователя, отправившего сообщение.

In [ ]:
#открываем файл с данными
internet = pd.read_csv('/datasets/internet.csv', index_col=0)

#вывод первых 5 строк таблицы
internet.head()

In [ ]:
internet.info()

**Таблица `internet`** состоит из 5 столбцов с типами данных `float64`(1), `int64`(2), `object`(2) и содержит информацию о 149396 интернет-сессиях:
* `id` — уникальный номер сессии;
* `mb_used` — объём потраченного за сессию интернет-трафика (в мегабайтах);
* `session_date` — дата интернет-сессии (день, месяц, год);
* `user_id` — идентификатор пользователя.

In [ ]:
#открываем файл с данными
tariffs = pd.read_csv('/datasets/tariffs.csv')

#вывод первых 5 строк таблицы
tariffs.head()

In [ ]:
tariffs.info()

**Таблица `tariffs`** состоит из 8 столбцов с типами данных `int64`(7), `object`(1) и содержит информацию о тарифах:
* `tariff_name` — название тарифа;
* `rub_monthly_fee` — ежемесячная абонентская плата (в рублях);
* `minutes_included` — количество минут разговора в месяц, включённых в абонентскую плату;
* `messages_included` — количество сообщений в месяц, включённых в абонентскую плату;
* `mb_per_month_included` — объём интернет-трафика, включённого в абонентскую плату (в мегабайтах);
* `rub_per_minute` — стоимость минуты разговора сверх тарифного пакета (например, если в тарифе 100 минут разговора в месяц, то со 101 минуты будет взиматься плата);
* `rub_per_message`— стоимость отправки сообщения сверх тарифного пакета;
* `rub_per_gb`— стоимость дополнительного гигабайта интернет-трафика сверх тарифного пакета (1 гигабайт = 1024 мегабайта).

По описаниям столбцов таблиц выше, есть такие (`reg_date`, `call_date`, `message_date`, `session_date`), в которых тип данных по факту `object`, но логичнее исправить на формат `datetime`, т.к. они содержат дату.

<div style="border:solid green 1px; padding: 10px">

В каждой таблице содержатся данные о пользователях: кто они, откуда, каким тарифом пользуются, сколько мегабайт интернета потратили, сколько сделали звонков и сообщений отправили за 2018 год.

Предварительно можно утверждать, что, данных достаточно для сравнения тарифов Компании.

Чтобы двигаться дальше, нужно устранить проблемы в данных и, возможно, объеденить таблицы в общий датафрейм.

## Предобработка данных

### Работа со столбцами и строками

Проверим уникальность пересечения значений первичных ключей (user_id) между объединяемыми таблицами для того, чтобы убедиться, что при объединении таблиц в одну, данные будут по всем 500 пользователям.

In [ ]:
# проверка
print(users['user_id'].isin(calls['user_id']).value_counts())
calls_user_id = calls['user_id']
ids_not_in_calls = users.query('user_id not in @calls_user_id')

Данных по 8ми пользователям из 500 нет в таблице звонков. Добавим недостоющих пользователей:

In [ ]:
# добавление строк в таблицу звонков
cntr = 0
while cntr != 8:
    for i in ids_not_in_calls['user_id']:
        calls = calls.append({'id':str(i)+'_0', 'call_date': '', 'duration': 0.00, 'user_id': i}, ignore_index=True)
    cntr += 1

In [ ]:
# проверка после обновления
users['user_id'].isin(calls['user_id']).value_counts()

In [ ]:
# проверка
print(users['user_id'].isin(messages['user_id']).value_counts())
messages_user_id = messages['user_id']
ids_not_in_msgs = users.query('user_id not in @messages_user_id')

Данных по 74м пользователям из 500 нет в таблице сообщений. Добавим недостоющих пользователей:

In [ ]:
# добавление строк в таблицу сообщений
cntr = 0
while cntr != 8:
    for i in ids_not_in_msgs['user_id']:
        messages = messages.append({'id':str(i)+'_0', 'message_date': '', 'user_id': i}, ignore_index=True)
    cntr += 1

In [ ]:
# проверка после обновления
users['user_id'].isin(messages['user_id']).value_counts()

In [ ]:
# проверка
print(users['user_id'].isin(internet['user_id']).value_counts())
inrnt_user_id = internet['user_id']
ids_not_in_inrnt = users.query('user_id not in @inrnt_user_id')

Данных по 3м пользователям из 500 нет в таблице интернет-трафика. Добавим недостоющих пользователей:

In [ ]:
# добавление строк в таблицу интернет-трафика
cntr = 0
while cntr != 8:
    for i in ids_not_in_inrnt['user_id']:
        internet = internet.append({'id':str(i)+'_0', 'mb_used': 0, 'session_date': '', 'user_id': i}, ignore_index=True)
    cntr += 1

In [ ]:
# проверка после обновления
users['user_id'].isin(internet['user_id']).value_counts()

In [ ]:
# изменеение типа данных
users['reg_date'] = pd.to_datetime(users['reg_date'], format='%Y.%m.%d')
calls['call_date'] = pd.to_datetime(calls['call_date'], format='%Y.%m.%d')
messages['message_date'] = pd.to_datetime(messages['message_date'], format='%Y.%m.%d')
internet['session_date'] = pd.to_datetime(internet['session_date'], format='%Y.%m.%d')

# заполнение пропусков
users['churn_date'] = users['churn_date'].fillna('-')

### Проверка на аномалии, исправления, дубликаты

In [ ]:
calls['duration'].sort_values().unique()

Есть значения "0,00" - это звонки с нулевой продолжительностью. Это не ошибка: нулями обозначены пропущенные звонки, поэтому их не нужно удалять.

In [ ]:
internet['mb_used'].sort_values().unique()

In [ ]:
internet['mb_used'].hist(bins=30);

In [ ]:
internet.query('mb_used == 0.0')['mb_used'].count() / internet.shape[0] * 100

Есть значения "0,00" и их доля в общем датафрейме чуть больше 13%. Если я все верно понимаю, при активной интернет-сессии трафик не может быть нулевым, строго говоря он идет даже если пользователь не входит в интернет от фоновой работы приложений телефона к примеру. Однако «Мегалайн» для веб-трафика отдельные сессии не считает. Вместо этого общая сумма за месяц округляется в бо́льшую сторону, а следовательно нулевые значения нам не помешают.

Дубликатов в данных из таблиц нет. Проверено методом `data.duplicated().sum()` по каждой таблице.

## Расчёты и добавление результатов в таблицу

Посчитаем для каждого пользователя:
* количество сделанных звонков и израсходованных минут разговора по месяцам;
* количество отправленных сообщений по месяцам;
* объем израсходованного интернет-трафика по месяцам;
* помесячную выручку с каждого пользователя.

### Количество сделанных звонков и израсходованных минут разговора по месяцам

In [ ]:
# добавим столбец месяца в таблицу звонков
calls['month'] = calls['call_date'].dt.month

# округляем время звонка согласно политике компании (в большую сторону)
calls['duration'] = np.ceil(calls['duration'])

# количество сделанных звонков и израсходованных минут разговора по месяцам 
calls_duration_month = (
    calls.pivot_table(index=['user_id', 'month'], values='duration', aggfunc=['count', 'sum'])
)

# переименование колонок
calls_duration_month.columns = ['count_calls','calls_duration']
calls_duration_month.head(3)

### Количество отправленных сообщений по месяцам

In [ ]:
# добавим столбец месяца в таблицу сообщений
messages['month'] = messages['message_date'].dt.month

# количество отправленных сообщений по месяцам 
messages_count_month = (
    messages.pivot_table(index=['user_id', 'month'], values='id', aggfunc='count')
)
messages_count_month.columns = ['count_messages']
messages_count_month.head(3)

### Объем израсходованного интернет-трафика по месяцам

In [ ]:
# добавим столбец месяца в таблицу интернет-трафикa
internet['month'] = internet['session_date'].dt.month

# объем израсходованного интернет-трафика по месяцам 
mb_used_month = (
    internet.pivot_table(index=['user_id', 'month'], values='mb_used', aggfunc='sum')
)
mb_used_month.columns = ['sum_mb_used']

# округляем трафик согласно политике компании (в большую сторону)
mb_used_month['sum_mb_used'] = np.ceil(mb_used_month['sum_mb_used'] / 1024) * 1024
mb_used_month.head(3)

### Сбор сводных таблиц с расчетами выше в один датафрейм для удобства визуализации и дальнейшей работы

In [ ]:
users_df_by_month = calls_duration_month.join([messages_count_month, mb_used_month], how='outer')
users_df_by_month.head()

In [ ]:
users_df_by_month.info()

In [ ]:
# наводим "красоту"
users_df_by_month = users_df_by_month.reset_index()

In [ ]:
users_df_by_month.head()

In [ ]:
# объединяем с таблицей по данным пользователей
users_full_df = users.merge(users_df_by_month, on='user_id', how='outer')

# объединяем с таблицей по данным о тaрифах
users_full_df['tariff_name'] = users_full_df['tariff']
del users_full_df['tariff']
users_full_df = users_full_df.merge(tariffs, on='tariff_name')
users_full_df.head(10)

In [ ]:
# проверяем пропуски
users_full_df.isna().sum()

In [ ]:
# заполняем пропуски
users_full_df['count_messages'] = users_full_df['count_messages'].fillna(0)
users_full_df['calls_duration'] = users_full_df['calls_duration'].fillna(0)
users_full_df['count_calls'] = users_full_df['count_calls'].fillna(0)
users_full_df['sum_mb_used'] = users_full_df['sum_mb_used'].fillna(0)
users_full_df.loc[243, 'month'] = users_full_df.loc[243, 'reg_date'].month
users_full_df.loc[2619, 'month'] = users_full_df.loc[2619, 'reg_date'].month

In [ ]:
users_full_df.isna().sum()

In [ ]:
# изменим тип данных для столбцов
columns_list = ['count_calls', 'calls_duration', 'count_messages', 'sum_mb_used', 'month']
for column in columns_list:
    users_full_df[column] =  users_full_df[column].astype('int')

### Помесячная выручка с каждого пользователя

In [ ]:
# считаем 
def minuts(row):
    tot_minuts = row['minutes_included'] - row['calls_duration']
    if tot_minuts < 0:
        tot_minuts = abs(tot_minuts) * row['rub_per_minute']
        return tot_minuts
    if tot_minuts >= 0:
        return 0

tot_minuts = users_full_df.apply(minuts, axis=1)

def msgs(row):
    tot_msgs = row['messages_included'] - row['count_messages']
    if tot_msgs < 0:
        tot_msgs = abs(tot_msgs) * row['rub_per_message']
        return tot_msgs
    if tot_msgs >= 0:
        return 0

tot_msgs = users_full_df.apply(msgs, axis=1)

def intrnt(row):
    tot_intrnt = row['mb_per_month_included'] - row['sum_mb_used']
    if tot_intrnt < 0:
        tot_intrnt = (abs(tot_intrnt) / 1024) * row['rub_per_gb'] + row['rub_monthly_fee']
        return tot_intrnt
    if tot_intrnt >= 0:
        return row['rub_monthly_fee']

tot_intrnt = users_full_df.apply(intrnt, axis=1)

prof = tot_minuts + tot_msgs + tot_intrnt
users_full_df['profit'] = prof

In [ ]:
# проверяем расчет по тарифу ultra
display(tariffs.query('tariff_name == "ultra"'))
users_full_df.query('tariff_name == "ultra"')[['calls_duration', 'count_messages', 'sum_mb_used', 'profit']].tail()

Все верно. Там, где не превышен бесплатный лимит, в столбце `profit` отображена только сумма абонентской платы по тарифу (к примеру строка №981). А там, где лимит превышен, разница домножена на тариф сверх бесплатного лимита (к примеру строка № 982: звонки и сообщения укладываются в лимит, а интернет превышен). 

In [ ]:
# проверяем расчет по тарифу smart
display(tariffs.query('tariff_name == "smart"'))
users_full_df.query('tariff_name == "smart"')[['calls_duration', 'count_messages', 'sum_mb_used', 'profit']].tail()

Все верно. Там, где не превышен бесплатный лимит в столбце `profit` отображена только сумма абонентской платы по тарифу (к примеру строка №3212). А там, где лимит превышен, разница домножена на тариф сверх бесплатного лимита (к примеру строка № 3214: превышены все три величины).

In [ ]:
# переведем интернет-трафик в ГБ
users_full_df['sum_gb_used'] = users_full_df['sum_mb_used'] / 1024

# соберем нужные столбцы
full_df = users_full_df[['user_id', 'age', 'churn_date', 'city', 'first_name', 'last_name',
       'reg_date', 'month', 'count_calls', 'calls_duration', 'count_messages',
       'sum_gb_used', 'tariff_name', 'profit']]

full_df.head()

In [ ]:
full_df['tariff_name'].value_counts()

В выборке из 500 пользователей, тариф `smart` у 2230 человек и тариф `ultra` у 986 человек.

## Анализ данных

### Предварительный анализ тарифов

Посмотрим сколько минут разговора, сколько сообщений и какой объём интернет-трафика требуется пользователям каждого тарифа в месяц. Посчитаем среднее количество, дисперсию и стандартное отклонение. Построим гистограммы.

In [ ]:
# вывод средних значений по тарифам в разрезе месяца, округлим до 2х знаков после запятой
mean_df = (
    round(full_df.pivot_table(index='month', columns='tariff_name',
                              values=['calls_duration', 'count_messages', 'sum_gb_used'],
                              aggfunc='mean'), 2)
)

In [ ]:
# выведем месяц в столбец
mean_df = mean_df.reset_index()

In [ ]:
mean_df

In [ ]:
std_var_df = full_df[['count_calls', 'calls_duration', 'sum_gb_used']]
# дисперсия
round(np.var(std_var_df), 2)

In [ ]:
# стандартное отклонение
round(np.std(std_var_df), 2)

In [ ]:
fig, calls_duration = plt.subplots()
calls_duration.hist(full_df.query('tariff_name == "smart"')['calls_duration'], alpha=0.5, label='smart')
calls_duration.hist(full_df.query('tariff_name == "ultra"')['calls_duration'], alpha=0.5, label='ultra')
calls_duration.set_title("Длительность звонков по тарифам")
calls_duration.set_xlabel('длительность звонков, мин')
calls_duration.legend()
calls_duration.grid()
calls_duration.xaxis.set_major_locator(ticker.MultipleLocator(100))
calls_duration.xaxis.set_minor_locator(ticker.MultipleLocator(50))
fig.set_figwidth(12)
fig.set_figheight(8)

In [ ]:
fig, count_messages = plt.subplots()
count_messages.hist(full_df.query('tariff_name == "smart"')['count_messages'], alpha=0.5, label='smart')
count_messages.hist(full_df.query('tariff_name == "ultra"')['count_messages'], alpha=0.5, label='ultra')
count_messages.set_title("Количество сообщений по тарифам")
count_messages.set_xlabel('количество сообщений, шт')
count_messages.legend()
count_messages.grid()
count_messages.xaxis.set_major_locator(ticker.MultipleLocator(20))
count_messages.xaxis.set_minor_locator(ticker.MultipleLocator(10))
fig.set_figwidth(12)
fig.set_figheight(8)

In [ ]:
fig, sum_mb_used = plt.subplots()
sum_mb_used.hist(full_df.query('tariff_name == "smart"')['sum_gb_used'], alpha=0.5, label='smart')
sum_mb_used.hist(full_df.query('tariff_name == "ultra"')['sum_gb_used'], alpha=0.5, label='ultra')
sum_mb_used.set_title("Интернет-трафик по тарифам")
sum_mb_used.set_xlabel('интернет-трафик, ГБ')
sum_mb_used.legend()
sum_mb_used.grid()
sum_mb_used.xaxis.set_major_locator(ticker.MultipleLocator(5))
sum_mb_used.xaxis.set_minor_locator(ticker.MultipleLocator(2))
fig.set_figwidth(12)
fig.set_figheight(8)

In [ ]:
mean_df.describe()

<div style="border:solid green 1px; padding: 10px">

По тарифу `smart` минимальные значения по минутам разговора, сообщениям и интернет-трафику пришлись на январь 2018 года, а по тарифу `ultra` на февраль. Максимальные значения по двум тарифам совпали и пришлись на декабрь года.

На графиках видно, что значения по тарифу `ultra` дальше "сдвинуты" вправо, чем по `smart` .

Пик значений примерно приходится:

1. по тарифу  `smart`:

* звонки в промежутке  430-550 минут
* сообщения в промежутке  0-10 шт
* трафик в промежутке  12-15 ГБ

2. по тарифу  `ultra`:

* звонки в промежутке  550-650 минут
* сообщения в промежутке  0-20 шт
* трафик в промежутке  15-20 ГБ

### Проверка гипотез

#### Средняя выручка пользователей тарифов «Ультра» и «Смарт»

Нужно провести сравнение выручки по тарифам и понять различается ли она. Определим формулировку для нулевой и альтернативной гипотезы:

Нулевая гипотеза - средние выручки пользователей тарифов «Ультра» и «Смарт»  равны.

Альтернативная гипотеза - средние выручки пользователей тарифов «Ультра» и «Смарт» не равны.

Пороговое значение alpha - критический уровень статистической значимости , примем как 0,05 (5%). Если p-value окажется меньше него - отвергнем гипотезу. P-value — это вероятность ошибки при отклонении нулевой гипотезы.

Для проверки гипотез используем столбец `prоfit`, т.к. в нем сохранены значения выручки по Клиентам.

In [ ]:
# проверка Но
alpha = 0.05
results = st.ttest_ind(
    full_df.query('tariff_name == "ultra"')['profit'], 
    full_df.query('tariff_name == "smart"')['profit']
)
print('p-значение: ', results.pvalue)

if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:

    print("Не получилось отвергнуть нулевую гипотезу")

In [ ]:
# посчитаем на сколько различается сумма выручки
full_df.query('tariff_name == "smart"')['profit'].sum() - full_df.query('tariff_name == "ultra"')['profit'].sum()

<div style="border:solid green 1px; padding: 10px">

Значение p-value 8.081909555489933e-161 - это практически 0, что говорит о том, что такое или большее различие практически не возможно получить случайно. Это дает возможность делать вывод о значимом различии в средней выручке пользователей по тарифам.

Суммарная выручка за год по тарифу "Смарт" почти на 835 тысяч больше чем по тарифу "Ультра".

Мы не использовали дополнительный параметр `equal_var = False`, потому что у нас достаточно большие и примерно одинаковые по размеру выборки, к тому же в обеих измерена одна и та же переменная — нет оснований полагать, что разброс будет сильно отличаться.

#### Средняя выручка пользователей из Москвы и других регионов

Нужно провести сравнение выручки в разрезе локации пользователей и понять различается ли она. Определим формулировку для нулевой и альтернативной гипотезы:

Нулевая гипотеза - средняя выручка пользователей из Москвы и пользователей из других регионов равны.

Альтернативная гипотеза - средняя выручка пользователей из Москвы и пользователей из других регионов не равны.

Пороговое значение alpha - критический уровень статистической значимости , примем как 0,05 (5%). Если p-value окажется меньше него - отвергнем гипотезу. P-value — это вероятность ошибки при отклонении нулевой гипотезы.

Для проверки гипотез используем столбец `prоft`, т.к. в нем сохранены значения выручки по Клиентам.

In [ ]:
full_df = full_df.reset_index(drop=True)

In [ ]:
# группировка по городу
def city_group(city):
    if city == 'Москва':
        return 'Москва'
    return 'Другой регион'

full_df['city_group'] = full_df['city'].apply(city_group)

In [ ]:
# проверка Но
results = st.ttest_ind(
    full_df.query('city_group == "Москва"')['profit'], 
    full_df.query('city_group != "Москва"')['profit'])
print('p-значение: ', results.pvalue)

if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:

    print("Не получилось отвергнуть нулевую гипотезу")

In [ ]:
# посчитаем на сколько различается сумма выручки
full_df.query('city_group != "Москва"')['profit'].sum() - full_df.query('city_group == "Москва"')['profit'].sum()

In [ ]:
# посчитаем на сколько различается сумма выручки по тарифу smart/ultra москвичей
(full_df.query('city_group == "Москва" and tariff_name == "smart"')['profit'].sum()
 - full_df.query('city_group == "Москва" and tariff_name == "ultra"')['profit'].sum())

In [ ]:
# посчитаем на сколько различается сумма выручки по тарифу smart/ultra не москвичей
(full_df.query('city_group != "Москва" and tariff_name == "smart"')['profit'].sum()
 - full_df.query('city_group != "Москва" and tariff_name == "ultra"')['profit'].sum())

<div style="border:solid green 1px; padding: 10px">

Полученное значение p-value говорит о том, что хотя средняя выручка пользователей из других регионов и из Москвы неодинакова, с вероятностью чуть больше 54% (что составляет больше половины случаев) такое или большее различие можно получить случайно.Такая вероятность случайно получить различие в значениях показателя, дает возможсть задуматься о возможном значимом различии в средней выручке пользователей из Москвы и из других регионов.

Суммарная выручка за год пользователей из других регионов превышает выручку пользователей из Москвы на 3 026 867,00. При этом и у пользователей из других регионов, и у пользователей из Москвы, по тарифу "Смарт" выручка больше чем по тарифу "Ультра" (на 823 109,00 и 11 742,00 соотвественно). 

Мы не использовали дополнительный параметр `equal_var = False`, потому что у нас достаточно большие и примерно одинаковые по размеру выборки, к тому же в обеих измерена одна и та же переменная — нет оснований полагать, что разброс будет сильно отличаться.

## <font color='green'>Общий вывод</font>

<div style="border:solid green 2px; padding: 20px">

    
Мы изучили выборку за 2018 год из 500 пользователей Компании «Мегалайн» для того чтобы скорректировать рекламный бюджет. Нужно было понять какой тариф приносит больше денег. Клиентам предлагается два тарифных плана: «Смарт» и «Ультра», также выборка содержит количество использованных минут разговора, отправленных сообщений и потраченного интернет-трафика и данные самих пользователях, такие как возвраст, город, дата начала/окончания пользования тарифом и пр.

    
Был проверен ряд параметров и выявлено, что база содержит куда больше данных по количеству пользователей тарифа "Смарт" чем по тарифу "Ультра", самым "не общительным" месяцем в году по тарифу "Смарт" стал январь, а по тарифу "Ультра" - февраль, а больше всего говорили, писали и выходили в интернет по двум тарифам в декабре.
    
В целом значения у тарифа "Ультра" больше, чем у тарифа "Смарт", это может говорить о том, что этот тариф предпочитают те пользователи, которым не хватает предлагаемого количества услуг из тарифа "Смарт".    

Пик значений приходится, примерно:

1. по тарифу  `smart`:

* звонки в промежутке  430-550 минут
* сообщения в промежутке  0-10 шт
* трафик в промежутке  12-15 ГБ
    
2. по тарифу  `ultra`:

* звонки в промежутке  550-650 минут
* сообщения в промежутке  0-20 шт
* трафик в промежутке  15-20 ГБ    
    
Однако по тарифу "Смарт" общее число пользователей, согласно выборке, больше чем у тарифа "Ультра": 2230 человек и 986 человек.
    
Были проверены две гипотезы и установлено, что:
    
* Средняя выручка пользователей тарифов «Ультра» и «Смарт» различаются. Суммарная выручка за год по тарифу "Смарт" почти на 835 тысяч больше чем по тарифу "Ультра".
* Средняя выручка пользователей из Москвы отличается от выручки пользователей из других регионов. Суммарная выручка за год пользователей из других регионов превышает выручку пользователей из Москвы на 3 026 867,00. При этом и у пользователей из других регионов, и у пользователей из Москвы, по тарифу "Смарт" выручка больше чем по тарифу "Ультра" (на 823 109,00 и 11 742,00 соотвественно). 
    
Итак, чтобы скорректировать рекламный бюджет следует принять во внимание, что в среднем по тарифу "Ультра" пользователи тратят больше минут на разговоры, пишут больше сообщений и тратят больше интернет-трафика, но таких пользователей меньше, большей популярностью пользуется тариф "Смарт". Он приносит больше денег вне зависимости от региона.